In [1]:
!pip install langchain_community
!pip install faiss-gpu
!pip install rank_bm25 
!pip install ragas
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.3
    Uninstalling langchain-text-splitters-0.3.3:
      Successfully uninstalled langchain-text-splitters-0.3.3
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
ERROR: 

In [2]:
import os
os.getcwd()

'/kaggle/working'

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline
from rank_bm25 import BM25Okapi
import numpy as np
import warnings

from datasets import Dataset
warnings.filterwarnings("ignore")
import textwrap

In [4]:
# 1. Load your text files
file_paths = ["/kaggle/input/knowledgebase/knowledgeBase.txt"]
documents = []

for file_path in file_paths:
    loader = TextLoader(file_path)
    docs = loader.load()
    documents.extend(docs)

# 2. Define chunking parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # You can try 100, 250, 512, etc.
    chunk_overlap=0       # Try 0, 50, 100, etc.
)

# 3. Split the documents
chunks = text_splitter.split_documents(documents)

# 4. Output result
print(f"Total chunks: {len(chunks)}")
print(f"First chunk content:\n{chunks[0].page_content}")

# Optional: Save the chunks to a file
with open("chunked_output.txt", "w", encoding='utf-8') as f:
    for i, chunk in enumerate(chunks):
        f.write(f"--- Chunk {i + 1} ---\n")
        f.write(chunk.page_content + "\n\n")


Total chunks: 114
First chunk content:
Faculty Data:
Dr. Aamir Sikander Kazi is a Consultant in the Medicine & Allied Services department, specializing in Infectious Diseases.
Dr. Abdul Hafeez is a Consultant in the Surgery & Allied Services department, specializing in Urology.
Dr. Abdul Rahim Ahmed is a Consultant in the Pediatric Services department, specializing in Pediatric Intensive Care.
Dr. Abdul Razzaque is a Consultant in the Imaging Services department, specializing in Radiology.


In [5]:
# Prepare documents and their metadata
texts = [chunk.page_content for chunk in chunks]
metadata = [chunk.metadata for chunk in chunks]
print(len(texts))

114


In [6]:
# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")

# Create FAISS vector database
vectordb = FAISS.from_documents(chunks, embedding_model)

# Save FAISS index to disk for later use
vectordb.save_local("faiss_index")

# Check the number of stored documents
print(f"Number of documents in the vector store: {vectordb.index.ntotal}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Number of documents in the vector store: 114


In [7]:
# BM25 Indexing
tokenized_texts = [text.split() for text in texts]
bm25 = BM25Okapi(tokenized_texts)

def reciprocal_rank_fusion(results_bm25, results_embedding, k=2):
    scores = {}

    # Use document content or metadata as the key
    for rank, (doc, score) in enumerate(results_bm25):
        doc_id = doc.page_content  # Or use doc.metadata.get("source", "unknown") if available
        scores[doc_id] = scores.get(doc_id, 0) + 1 / (rank+1) # (k + rank + 1)
        print("BM25", scores[doc_id])

    for rank, (doc, score) in enumerate(results_embedding):
        doc_id = doc.page_content  # Use the same identifier
        scores[doc_id] = scores.get(doc_id, 0) + 1 / (rank+1) # (k + rank + 1)
        print("Dense", scores[doc_id])

    return sorted(scores.items(), key=lambda x: x[1], reverse=True)


# Extract page content and metadata properly
def format_response(doc):
    return f"Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content.strip()}"

In [8]:
# Retrieve function
def retrieve(query, k=3):
    query_embedding = embedding_model.embed_query(query)
    results_embedding = vectordb.similarity_search_with_score_by_vector(query_embedding, k=k)
    results_embedding = sorted(results_embedding, key=lambda x: x[1], reverse=True)
    
    print("============Dense Embeddings=============")
    for doc, score in results_embedding:
        print(f"page {doc.metadata.get('page','Unknown')} - Score: {score:.4f} - {doc.page_content[:100]}...")

    # Get BM25 scores for all documents and sort to get top-k results
    results_bm25 = [(idx, bm25.get_scores(query.split())[idx]) for idx in range(len(texts))]
    results_bm25 = sorted(results_bm25, key=lambda x: x[1], reverse=True)[:k]  # Keep only top-k results
    # Convert BM25 results to (Document, score) format
    results_bm25_docs = [(Document(page_content=texts[idx], metadata=metadata[idx]), score) for idx, score in results_bm25]
   
    print("************BM25 Results*************")
    for doc, score in results_bm25_docs:
        print(f"page {doc.metadata.get('page','Unknown')} - Score: {score:.4f} - {doc.page_content[:100]}...")
    
    # Create a lookup dictionary {document content -> Document object}
    doc_lookup = {doc.page_content: doc for doc, _ in results_bm25_docs}
    doc_lookup.update({doc.page_content: doc for doc, _ in results_embedding})

    # Fuse results
    fused_results = reciprocal_rank_fusion(results_bm25_docs, results_embedding)
    
    # Format results, ensuring document IDs are mapped back to actual Documents
    return [format_response(doc_lookup[doc_id]) for doc_id, _ in fused_results if doc_id in doc_lookup]

    #fused_results = reciprocal_rank_fusion(results_bm25, results_embedding)
    #return [(texts[idx], metadata[idx]["page"] if "page" in metadata[idx] else "Unknown") for idx, _ in fused_results]


In [9]:
# Query example
question = "Where are the blood collection centers in Karachi?"
retrieved_responses = retrieve(question, k=3)

============Dense Embeddings=============
page Unknown - Score: 0.2777 - Donate Online
Make donations directly using Visa/MasterCard through the HBL Payment Gateway.

Collec...
page Unknown - Score: 0.2440 - With the idea of community-based interventions, this program encompasses.

Facility-based Family Med...
page Unknown - Score: 0.2130 - Mobile Wallet
Donate using JazzCash, EasyPaisa, or UPaisa accounts.

Donation Box
Place a Donation B...
************BM25 Results*************
page Unknown - Score: 10.1028 - Children’s data: While we do not actively collect information from children (under 18s), we apprecia...
page Unknown - Score: 9.8612 - Since the start of the voluntary blood donation program in December 2013, the Indus Hospital Blood C...
page Unknown - Score: 9.7137 - Donate Online
Make donations directly using Visa/MasterCard through the HBL Payment Gateway.

Collec...
BM25 1.0
BM25 0.5
BM25 0.3333333333333333
Dense 1.3333333333333333
Dense 0.5
Dense 0.3333333333333333


In [10]:
# Query processing
question = "Where are the blood collection centers in Karachi?"
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
docs = retriever.get_relevant_documents(question)

# Print results
for i, doc in enumerate(docs, 1):
    page_number = doc.metadata.get('page', 'Unknown')
    # print(f"Document {i} - Page {page_number} - Score: {doc.metadata.get('score', 'N/A')}")
    print(doc.page_content[:])  # Print first 500 characters of each result
    print("-" * 80)

Mobile Wallet
Donate using JazzCash, EasyPaisa, or UPaisa accounts.

Donation Box
Place a Donation Box at your retail outlet.
Call:

Karachi: 021 111 111 880

Lahore: 042 111 111 880

UAN: 03 111 111 880 880

Blood Collection Centers: 
Karachi
1. Korangi Crossing
Address: Indus Hospital-Korangi Campus, Plot C-76 Sector 31/5, Opposite Darussalam Society Korangi Crossing, Karachi
Phone: 021 3511 2710-17

Normal Days:
Mon to Fri: 9:00 am to 5:30 pm
Sat: 9:00 am to 1:30 pm
Sun: Closed
--------------------------------------------------------------------------------
With the idea of community-based interventions, this program encompasses.

Facility-based Family Medicine Clinics

Mobile Medical Bus

Container-based Community Health Centers

Boat Clinics

Telehealth

Blood Center
The Indus Hospital Blood Center (TIHBC), Karachi, was established in 2013 and is Pakistan's first centralized blood center. The goals of TIHBC are to mobilize the country for 100% voluntary blood donations while provi

In [11]:

model_name = "Qwen/Qwen2.5-3B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto", #device_map='cuda'
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [12]:
# Create a pipeline
generator = pipeline(
"text-generation",
model=model,
tokenizer=tokenizer,
return_full_text=False,
max_new_tokens=5000,
do_sample=False
)

Device set to use cuda:0


In [13]:
# # Extract page content and metadata properly
# def format_response(doc):
#     return f"Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content.strip()}"

# # Handle cases where fewer than 3 results are returned
# retrieved_responses = [format_response(doc) for doc in docs[:3]]
# while len(retrieved_responses) < 3:
#     retrieved_responses.append("No relevant information.")  # Fill missing slots

In [14]:
# Construct the RAG prompt
prompt = f"""
You are an AI assistant tasked with answering questions based on retrieved knowledge.

### **Retrieved Information**:
1. {retrieved_responses[0]}

2. {retrieved_responses[1]}

3. {retrieved_responses[2]}

### **Question**:
{question}

### **Instructions**:
- Integrate the key points from all retrieved responses into a **cohesive, well-structured answer**.
- If the responses are **contradictory**, mention the different perspectives.
- If none of the retrieved responses contain relevant information, reply:
  **"I couldn't find a good response to your query in the database."**
"""



In [15]:
for i in range(0,len(retrieved_responses)):
    print(retrieved_responses[i])
    print("-------")

Page Unknown: Donate Online
Make donations directly using Visa/MasterCard through the HBL Payment Gateway.

Collection Centers
Visit collection centers in Karachi, Lahore & Hyderabad.


Donate Via SMS
Type DONATE and send an SMS to 9880
(Charges: Rs. 20 + Tax)

Bank App
Contribute through your bank’s mobile app.

Daraz
Visit the Daraz website to make an online contribution.

Doorstep Collection

Karachi: 021 111 111 880

Lahore: 042 111 111 880

UAN: 03 111 111 880
-------
Page Unknown: Children’s data: While we do not actively collect information from children (under 18s), we appreciate that our supporters are of all ages. Where appropriate, we will always ask for consent from a parent or guardian to collect information about children. All Indus Hospital & Health Network events will have clear rules on whether or not children can take part, and the collection of data will be managed in accordance with each individual event, with appropriate safeguards in place.
-------
Page Unknown: S

In [16]:
# Use Qwen2.5 3B with the correct message format
messages = [
    {"role": "user", "content": prompt}
]

# Generate output using the model
output = generator(messages)

# Print formatted response
print(textwrap.fill(output[0]["generated_text"], width=80))

Based on the information provided, there is no specific mention of blood
collection centers in Karachi. The retrieved information primarily focuses on
donation methods such as online donations, SMS donations, bank contributions,
and a doorstep collection service. However, it does provide contact numbers for
doorstep collections in Karachi:  - Karachi: 021 111 111 880  Unfortunately, I
couldn't find detailed information about blood collection centers specifically
in Karachi within the given set of responses. For accurate and up-to-date
information regarding blood collection centers, it would be best to consult the
official Indus Hospital & Health Network's website or contact them directly.


In [17]:
# from ragas.metrics import answer_relevancy
# from ragas import evaluate

# data = {
#     "question": [question],
#     'retrieved_contexts' : [[""]],
#     # "contexts": [["Dr. Abdul Hafeez is a Consultant in the Surgery department, specializing in Urology."]],
#     "ground_truth": ["Dr. Aneela Hussain and Dr. Aamir Sikander Kazi meet the criteria of being a Consultant in the Medicine & Allied Services department, specializing in Infectious Diseases."],
#     "answer": [textwrap.fill(output[0]["generated_text"], width=80)]
# }



In [18]:
# # Convert to HuggingFace Dataset
# dataset = Dataset.from_dict(data)

# # Evaluate both faithfulness and relevancy
# results = evaluate(dataset, metrics=[answer_relevancy, faithfulness])

# # Print the results
# df = results.to_pandas()
# print(df)